---

# 4. Analysis of Context-free Languages
**[Emil Sekerinski](http://www.cas.mcmaster.ca/~emil/), McMaster University, January 2026**

---

### Pushdown Automata

Context-free languages can contain nested structures, e.g.

    S → a S c | b

Recognizing this language requires matching an unbounded number of `a` symbols with the same number of `c` symbols, which finite state automata cannot do.

Context-free languages can be recognized by _pushdown automata_ that operate on a stack: transitions can push on and pop from the stack. The stack size is not bounded.

A pushdown automaton `P = (Σ, S, R, s₀)` is specified by

- a finite set `Σ` of *input symbols*,
- a finite set `S` of *stack symbols*,
- a finite set `R` of *transitions*,
- an _initial stack symbol_ `s₀ ∈ S ∪ {ε}`,

where `Σ` is also called the _vocabulary_ and each transition is a triple with a sequence `σ ∈ S*`, an input symbol `a ∈ Σ ∪ {ε}`, and a sequence `σ' ∈ S*`, written:

	  σ a → σ'

The pushdown automaton starts with just `s₀` on the stack. A transition `σ a → σ'` can be taken if the top of the stack is `σ` and `t` can be consumed from the input: the transition will pop `σ` from the stack and push `σ'` on the stack. If there are no more input symbols when the stack is empty, the input string is accepted; otherwise, it is rejected.

<div style="float:right;border-left:4em solid white">

| transition | stack | input         |
|:-----------|------:|--------------:|
| ` `        | `ε`   | `a a b b b a` |
| (1)        | `a`   | `a b b b a`   |
| (1)        | `a a` | `b b b a`     |
| (4)        | `a`   | `b b a`       |
| (4)        | `ε`   | `b a`         |
| (3)        | `b`   | `a`           |
| (2)        | `ε`   | `ε`           |

</div>

For example, an automaton accepting sequences over `Σ = {a, b}` with the same number of occurrences of `a` and `b`, formally `{τ ∈ Σ* | a#τ = b#τ}`, is `P₀ = (Σ, S, R, s₀)` where `S = {a, b}`, `s₀ = ε`, and the transitions `R` are:

  `ε a → a` <span style="float:right">(1)</span>  
  `b a → ε` <span style="float:right">(2)</span>  
  `ε b → b` <span style="float:right">(3)</span>  
  `a b → ε` <span style="float:right">(4)</span>

Here, the stack symbols coincide with the input symbols. The input `a a b b b a` is accepted by `P₀` as in the table. By convention, the top of the stack is to the left.

<div style="float:right;border-left:4em solid white">

| transition | stack | input       |
|:-----------|------:|:------------|
| ` `        | `s`   | `a a b c c` |
| (1)        | `s.`  | `a b c c`   |
| (1)        | `s..` | `b c c`     |
| (2)        | `..`  | `c c`       |
| (3)        | `.`   | `c`         |
| (3)        | `ε`   | `ε`         |

</div>

As another example, the pushdown automaton for accepting the language generated by `S → a S c | b` is `P₁ = (Σ, S, R, s₀)` where `Σ = {a, b, c}`, `S = {s, .}`, `s₀ = s`, and the transitions `R` are:

  `s a → s.` <span style="float:right">(1)</span>  
  `s b → ε` <span style="float:right">(2)</span>  
  `. c → ε` <span style="float:right">(3)</span>

The input `a a b c c` is accepted as in the table.

**Exercise:** What is the pushdown automaton for accepting palindromes over `{a, b, c}`, i.e. sequences that read backward the same as forward?

For every finite state automaton, an equivalent pushdown automaton can be constructed. For example, the finite state automaton accepting regular expression `E₂ = a b | a c` is `A₂ = (Σ, Q, I, δ, F)` with `Σ = {a, b, c}`, `Q = {0, 1, 2, 3, 4}`, `I = {0}`, `F = {3, 4}`, and transitions `δ`:

    0 a → 1
    0 a → 2
    1 b → 3
    2 c → 4

The equivalent pushdown automaton `P₂ = (Σ, S, R, s₀)` has the same vocabulary `Σ`, has `S = {0, 1, 2}`, `s₀ = 0`, and has transitions `R`:

  `0 a → 1` <span style="float:right">(1)</span>  
  `0 a → 2` <span style="float:right">(2)</span>  
  `1 b → ε` <span style="float:right">(3)</span>  
  `2 c → ε` <span style="float:right">(4)</span>

<div style="float:right;border-left:4em solid white">

| transition | stack | input  |
|:-----------|------:|:-------|
| ` `        | `0`   | `a b`  |
| (1)        | `1`   | `b`    |
| (3)        | `ε`   | `ε`    |

</div>

That is, the stack is initialized with the initial state of `A₂`, the transitions of `A₂` and `P₂` are the same, except that transitions in `A₂` to final states pop the state from the stack in `P₁`. The input `a b` is accepted as in the table.

For every context-free grammar, an equivalent pushdown automaton can be constructed and vice versa.

As with finite state automata, pushdown automata can be either deterministic or nondeterministic. Unlike finite state automata, it is generally impossible to make a pushdown automaton deterministic and run in linear time. The best one can achieve in general is to accept in approximately `n³` time, where `n` is the length of the input.

For accepting in linear time, restrictions on the languages must be imposed and, therefore, on the grammars generating these languages. There are different ways to construct a pushdown automaton given a grammar, each with different restrictions on the grammar. Since the ultimate goal is to determine the meaning of a sentence through its parse tree and not just to accept it, we must be careful with grammar modifications to suit the construction of the pushdown automaton.

### Top-down and Bottom-up Parsing

<div style="float:right;border-left:4em solid white">

| step   | stack     | input         |
|:-------|----------:|:--------------|
|        | `E`       | `x × (y + z)` |
| P (1)  | `T`       | `x × (y + z)` |
| P (4)  | `T × F`   | `x × (y + z)` |
| P (3)  | `F × F`   | `x × (y + z)` |
| P (5)  | `id × F`  | `x × (y + z)` |
| M (7)  | `× F`     | `× (y + z)`   |
| M (9)  | `F`       | `(y + z)`     |
| P (6)  | `(E)`     | `(y + z)`     |
| M (10) | `E)`      | `y + z)`      |
| P (2)  | `E + T)`  | `y + z)`      |
| P (1)  | `T + T)`  | `y + z)`      |
| P (3)  | `F + T)`  | `y + z)`      |
| P (5)  | `id + T)` | `y + z)`      |
| M (7)  | `+ T)`    | `+ z)`        |
| M (8)  | `T)`      | `z)`          |
| P (3)  | `F)`      | `z)`          |
| P (5)  | `id)`     | `z)`          |
| M (7)  | `)`       | `)`           |
| M (11) | ` `       | ` `           |

</div>

_Top-down parsing_ starts to build the parse tree with the start symbol as the goal, which is split into subgoals for each non-terminal according to the grammar rules until the terminals match the input.

Consider parsing the sentence `x × (y + z)` with grammar `G₃`:

    E → T | E + T
    T → F | T × F
    F → id | ( E )

The equivalent top-down pushdown automaton `P₃ = (Σ, S, R, s₀)` has the same vocabulary `Σ = {+, ×, id, (, )}`, has stack symbols `S = {E, T, F, +, ×, id, (, )}`, `s₀ = E`, and has transitions `R`:

  `E ε → T` <span style="float:right">(1)</span>  
  `E ε → E + T` <span style="float:right">(2)</span>  
  `T ε → F` <span style="float:right">(3)</span>  
  `T ε → T × F` <span style="float:right">(4)</span>  
  `F ε → id` <span style="float:right">(5)</span>  
  `F ε → ( E )` <span style="float:right">(6)</span>  
  `id id → ε` <span style="float:right">(7)</span>  
  `+ + → ε` <span style="float:right">(8)</span>  
  `× × → ε` <span style="float:right">(9)</span>  
  `( ( → ε` <span style="float:right">(10)</span>  
  `) ) → ε` <span style="float:right">(11)</span>

A top-down parser takes *produce (expand) steps* for transitions (1) to (6) and *match steps* for transitions (7) to (11).

Because of the direct correspondence between a context-free grammar and its bottom-up pushdown automaton, we omit from now on the explicit definition of the automaton.

<div style="float:right;border-left:4em solid white">

| step   | stack         | input           |
|:-------|--------------:|:----------------|
|        |               | `x × ( y + z )` |
| S (7)  | `x`           | `× ( y + z )`   |
| R (5)  | `F`           | `× ( y + z )`   |
| R (3)  | `T`           | `× ( y + z )`   |
| S (9)  | `× T`         | `( y + z )`     |
| S (10) | `( × T`       | `y + z )`       |
| S (7)  | `y ( × T`     | `+ z )`         |
| R (5)  | `F ( × T`     | `+ z )`         |
| R (3)  | `T ( × T`     | `+ z )`         |
| R (1)  | `E ( × T`     | `+ z )`         |
| S (8)  | `+ E ( × T`   | `z )`           |
| S (7)  | `z + E ( × T` | `)`             |
| R (5)  | `F + E ( × T` | `)`             |
| R (3)  | `T + E ( × T` | `)`             |
| R (2)  | `E ( × T`     | `)`             |
| S (11) | `) E ( × T`   |                 |
| R (6)  | `F × T`       |                 |
| R (3)  | `T`           |                 |
| R (1)  | `E`           |                 |

</div>

_Bottom-up parsing_ proceeds without a specific goal; the parse tree grows from bottom to top; the input is accepted if it is reduced to the start symbol by two kinds of steps:

- *Shift steps* shift the next input symbol on the stack.
- *Reduce steps* reduce a sequence of symbols on the stack according to a transition.

Consider parsing the sentence `x × (y + z)` with grammar `G₃`:

    E → T | E + T
    T → F | T × F
    F → id | ( E )

The equivalent bottom-up pushdown automaton `P₃' = (Σ, S, R, s₀)` has the same vocabulary `Σ = {+, ×, id, (, )}`, has stack symbols `S = {E, Σ, F, +, ×, id, (, )}`, `s₀ = ε`, and has transitions `R`:

  `T ε → E` <span style="float:right">(1)</span>  
  `T + E ε → E` <span style="float:right">(2)</span>  
  `F ε → T` <span style="float:right">(3)</span>  
  `F × T ε → T` <span style="float:right">(4)</span>  
  `id ε → F` <span style="float:right">(5)</span>  
  `) E ( ε → F` <span style="float:right">(6)</span>  
  `ε id → id` <span style="float:right">(7)</span>  
  `ε + → +` <span style="float:right">(8)</span>  
  `ε × → ×` <span style="float:right">(9)</span>  
  `ε ( → (` <span style="float:right">(10)</span>  
  `ε ) → )` <span style="float:right">(11)</span>

The parser takes shift steps for transitions (7) to (11) and reduce steps for transitions (1) to (6).

As in pushdown automata, an input is accepted if the stack is empty; this can be achieved by adding one more transition, `E ε → ε`.

### `LL(k)` and `LR(k)`

Consider the *leftmost* and *rightmost* derivations of `x × (y + z)`:

<div style="float:left;border-left:4em solid white">

    E ⇒ T
       ⇒ T × F
       ⇒ F × F
       ⇒ x × F
       ⇒ x × ( E )
       ⇒ x × ( E + T )
       ⇒ x × ( T + T )
       ⇒ x × ( F + T )
       ⇒ x × ( y + T )
       ⇒ x × ( y + F )
       ⇒ x × ( y + z )
</div>

<div style="float:left;border-left:4em solid white">

    E ⇒ T
       ⇒ T × F
       ⇒ T × ( E )
       ⇒ T × ( E + T )
       ⇒ T × ( E + F )
       ⇒ T × ( E + z )
       ⇒ T × ( T + z )
       ⇒ T × ( F + z )
       ⇒ T × ( y + z )
       ⇒ F × ( y + z )
       ⇒ x × ( y + z )

</div>

We denote these derivations by `⇒ᴸ` and `⇒ᴿ`, respectively. In top-down parsing, the leftmost symbol is expanded, as in `⇒ᴸ`. In bottom-up parsing, the concatenation of the stack and the input is `⇒ᴿ` in reverse order.

The method `L` of class `Grammar` is modified to generate `Lᴸ(G) = {χ ∈ T* | S ⇒ᴸ χ}` and `Lᴿ(G) = {χ ∈ T* | S ⇒ᴿ χ}`.

In [ ]:
from collections.abc import Iterator

class Grammar:
    def __init__(self, T: set[str], N: set[str], P: set[tuple[str, str]], S: str):
        self.T, self.N, self.P, self.S = T, N, P, S
    def LL(self, log = False, stats = False) -> Iterator[str]:
        dd, d = set(), {self.S}
        if log: print('    ', self.S)
        while d != set():
            if stats: print('# added derivations:', len(d))
            if log: print()
            dd.update(d); d = set()
            for π in sorted(dd, key = len):
                l = min((π.find(x) for x in π.split() if x in self.N), default = len(π))
                for σ, τ in self.P: # production σ → τ
                    i = π.find(σ)
                    if i != - 1 and i == l: # π == π[0:i] + σ + π[i + len(σ):]
                        χ = π[0:i] + τ + π[i + len(σ):]; χ = χ.replace('  ', ' ')
                        if (χ not in dd) and (χ not in d):
                            if all(a in self.T for a in χ.split()): yield χ.strip()
                            if log: print('    ', π, '⇒ᴸ', χ)
                            d.add(χ)
    def LR(self, log = False, stats = False) -> Iterator[str]:
        dd, d = set(), {self.S}
        if log: print('    ', self.S)
        while d != set():
            if stats: print('# added derivations:', len(d))
            if log: print()
            dd.update(d); d = set()
            for π in sorted(dd, key = len):
                l = max((π.rfind(x) for x in π.split() if x in self.N), default = -1)
                for σ, τ in self.P: # production σ → τ
                    i = π.rfind(σ)
                    if i != - 1 and i == l: # π == π[0:i] + σ + π[i + len(σ):]
                        χ = π[0:i] + τ + π[i + len(σ):]; χ = χ.replace('  ', ' ')
                        if (χ not in dd) and (χ not in d):
                            if all(a in self.T for a in χ.split()): yield χ.strip()
                            if log: print('    ', π, '⇒ᴿ', χ)
                            d.add(χ)

For example:

In [ ]:
G3 = Grammar({'x', 'y', 'z', '+', '×', '(', ')'}, {'E', 'T', 'F'},
             {('E', 'T'), ('E', 'E + T'), ('T', 'F'), ('T', 'T × F'),
              ('F', 'x'), ('F', 'y'), ('F', 'z'), ('F', '( E )')},
             'E')

In [ ]:
[d for d, _ in zip(G3.LL(), range(10))]

Method `derivableL` produces the leftmost derivation:

In [ ]:
def derivableL(G: Grammar, ω: str, log = False, stats = False) -> bool: # G must be context-sensitive
    dd, d, ω = set(), {G.S}, ω.strip()
    if log: print('    ', G.S)
    while d != set():
        if stats: print('# added derivations:', len(d))
        if log: print()
        dd.update(d); d = set()
        for π in sorted(dd, key = len):
            l = min((π.find(x) for x in π.split() if x in G.N), default = len(π))
            for σ, τ in G.P: # production σ → τ
                i = π.find(σ)
                if i != - 1 and i == l: # π == π[0:i] + σ + π[i + len(σ):]
                    χ = π[0:i] + τ + π[i + len(σ):]; χ = χ.replace('  ', ' ')
                    if (χ not in dd) and (χ not in d):
                        if χ.strip() == ω: return True
                        elif len(χ.strip()) <= len(ω):
                            if log: print('    ', π, '⇒ᴸ', χ)
                            d.add(χ)
    return False

setattr(Grammar, 'derivableL', derivableL)

For example:

In [ ]:
assert G3.derivableL('x + y', log = True, stats = True)

In [ ]:
assert G3.derivableL('x × ( y + z )', stats = True)

Method `derivableR` produces the rightmost derivation:

In [ ]:
def derivableR(G: Grammar, ω: str, log = False, stats = False) -> bool: # G must be context-sensitive
    dd, d, ω = set(), {G.S}, ω.strip()
    if log: print('    ', G.S)
    while d != set():
        if stats: print('# added derivations:', len(d))
        if log: print()
        dd.update(d); d = set()
        for π in sorted(dd, key = len):
            l = max((π.rfind(x) for x in π.split() if x in G.N), default = -1)
            for σ, τ in G.P: # production σ → τ
                i = π.rfind(σ)
                if i != - 1 and i == l: # π == π[0:i] + σ + π[i + len(σ):]
                    χ = π[0:i] + τ + π[i + len(σ):]; χ = χ.replace('  ', ' ')
                    if (χ not in dd) and (χ not in d):
                        if χ.strip() == ω: return True
                        elif len(χ.strip()) <= len(ω):
                            if log: print('    ', π, '⇒ᴿ', χ)
                            d.add(χ)
    return False

setattr(Grammar, 'derivableR', derivableR)

For example:

In [ ]:
assert G3.derivableR('x + y', log = True, stats = True)

In [ ]:
assert G3.derivableR('x × ( y + z )', stats = True)

In both top-down and bottom-up parsing, the input is read from left to right. In both methods, there is nondeterminism in the selection of the next step. For deterministic parsing, the decision for the next step can be restricted to be based on the stack and up to `k` symbols of lookahead. The grammars suitable for deterministic top-down and bottom-up parsing methods are known as:

- `LL(k)`: left-to-right parse, leftmost derivation, `k` symbol lookahead
- `LR(k)`: left-to-right parse, rightmost derivation, `k` symbol lookahead

Let `k:ω` be `ω` truncated to the first `k` symbols. Consider grammar `G = (T, N, P, S)`.

**Definition.** `G` is `LL(k)` if for arbitrary derivations
```
                        ⇗  μ ν χ ⇒* μ γ
    S ⇒ᴸ μ A χ                                           μ, γ, γʹ ∈ T*, ν, νʹ, χ ∈ V*, A ∈ N
                        ⇘  μ νʹ χ ⇒* μ γʹ
```
`k:γ = k:γ'` implies `ν = ν'`.

**Definition.** `G` is `LR(k)` if for arbitrary derivations
```
        ⇗ᴿ  μ A ω ⇒ μ χ ω
    S                                           ω, ωʹ ∈ T*, μ ∈ V*, A ∈ N
        ⇘ᴿ  μ Aʹ ωʹ ⇒ μ χ ωʹ
```
`k:ω = k:ω'` implies `A = A'`.

Consider the following EBNF grammar for statements:

    S → I ':=' E | I '(' E ')'
    I → id [ '.' id]

This grammar is none of `LL(1)`, `LL(2)`, `LL(3)`, but is `LL(4)` and `LR(1)`. Consider now:

    S → I ':=' E | I '(' E ')'
    I → id { '.' id }

This grammar is not `LL(k)` for any `k`, but is `LR(1)`. However, it could be rewritten to be `LL(1)`. In general, this is not possible: More languages can be generated by `LR(1)` grammars than by `LL(1)` grammars.

**Theorem.**  
- For every `LR(k)` grammar there exists an equivalent `LR(1)` grammar.
- Every `LL(k)` grammar is also an `LR(k)` grammar.
- There exists `LR(k)` grammars that are not `LL(k')`, for any `k'`.

The definitions of `LL(k)` and `LR(k)` are in terms of arbitrary derivations. Since there can be infinitely many, these definitions don't lead to a direct way of determining if a grammar is `LL(k)` or `LR(k)`. The `LL(k)` restriction can also be checked in terms of the grammar. For `LR(k)`, no simple checks exist; instead, a pushdown automaton has to be constructed and if that succeeds, the grammar is `LR(k)`.

### Conditions for Top-down Parsing

<div style="float:right;border-left:2em solid white">

| step | stack | input     |
|:-----|------:|:----------|
| ` `  | `S`   | `x x x z` |
| P    | `A`   | `x x x z` |
| P    | `x A` | `x x x z` |
| M    | `A`   | `x x z`   |
| P    | `x A` | `x x z`   |
| M    | `A`   | `x z`     |
| P    | `x A` | `x z`     |
| M    | `A`   | `z`       |
    
</div>

We continue with top-down parsing, also called _predictive parsing_, as we have to predict which production to expand at each P step, and restrict to one symbol lookahead, i.e., `LL(1)`.

Consider parsing `x x x z` in grammar `G₃`:

    S → A | B
    A → x A | y
    B → x B | z

After an unfortunate initial P step, we get stuck. One would need to look ahead to the last input symbol to prevent that. However, with this grammar, there is no bound on the number of symbols one would need to look ahead in general.

The required restrictions on the grammar are expressed in terms of the _first_ and _follow sets_.

The set `first(ω)` is the set of all terminals that can appear in the first position of sentences derived from `ω`:

    first(ω) = {t ∈ T | ω ⇒* t ν, for some ν ∈ V*}
	
The set `follow(ω)` is the set of all terminal symbols that may follow `ω` in any sentence:

    follow(ω) = {t ∈ T | S ⇒* μ ω t ν, for some μ, ν ∈ V*}

For example, in `G₃` with `S → A | B`, `A → x A | y`, `B → x B | z` we have:

- `first(A) = {x, y}`, `first(B) = {x, z}`, `first(S) = {x, y, z}`, `first(xA) = {x}`
- `follow(x) = {x, y, z}`, `follow(xA) = {}`, `follow(A) = {}`

Two conditions are required to ensure that a deterministic, one-symbol lookahead top-down parser can be constructed.

**Condition 1.** If `A` is defined by the production

    A → χ₁ | χ₂ | …

then the initial symbols of all sentences that can be derived from all `χᵢ` must be distinct, i.e.:

  `first(χᵢ) ∩ first(χⱼ) = ∅`  for all  `i ≠ j`

For example, for `G₃` with  `S → A | B`, `A → x A | y`, `B → x B | z` we have that `first(A) = {x, y}` and `first(B) = {x, z}`. Production `S → A | B` does not satisfy Condition 1. An equivalent grammar with production `S → x S | y | z` satisfies the condition.

<div style="float:right;border-left:2em solid white">

| step | stack | input |
|:-----|------:|:------|
| ` `  | `S`   | `x`   |
| P    | `A x` | `x`   |
| P    | `x x` | `x`   |
| M    | `x`   |       |

</div>

Consider parsing `x` in grammar `G₄`; we again may get stuck:

	S → A x
    A → x | ε

**Condition 2.** For every nonterminal `A` from which the empty sequence can be derived, the set of initial symbols must be disjoint from the set of symbols that may follow any sequence generated from `A`:

  `first(A) ∩ follow(A) = ∅`  for all `A` such that `A ⇒* ε`

If `A ⇒* ε`, then `A` is called _nullable_.

For example, in `G₄` with `S → A x`, `A → x | ε` we have:

- `first(A) = {x} = follow(A)`
- `A ⇒* ε`

Hence, Condition 2 is violated.

**Theorem**. Grammar `G = (T, N, P, S)` is `LL(1)` if for any distinct productions `A → χ` and `A → χʹ`:

    first(χ follow (A)) ∩ first(χʹ follow(A)) = ∅

Conditions 1 and 2 above are equivalent to this theorem by distinguishing the cases if `A` is nullable and not.

### Recursive Descent Parsing

The appeal of top-down parsing is that an acceptor can be directly expressed in a programming language with mutually recursive procedures by a parsing technique known as _recursive descent_. There is no need to represent the stack of the pushdown automaton explicitly; the stack of the programming language is sufficient. Recursive descent parsing assumes that the grammar is in EBNF.

For each production `p`, a parsing procedure `Pr p` is constructed. For production `B → E`, the name of the procedure is `B`, and its body is `Pr E`, a parser recognizing EBNF expression `E`:

| `p`             | `Pr p`                    |
|:----------------|:--------------------------|
| `B → E`         | `procedure B` <br> `Pr E` |

Assume that procedure `next` reads and assigns the next input symbol to global variable `sym`. The rules for constructing `Pr E` for recognizing `E` are:

| `E`             | `Pr E`                              |
|:----------------|:------------------------------------|
| `'a'`           | `if sym = 'a' then next else error` |
| `B`             | `B`                                 |
| `(E₁)`          | `Pr E₁`                             |
| `[E₁]`          | `if sym ∈ first(E₁) then Pr E₁`     |
| `{E₁}`          | `while sym ∈ first(E₁) do Pr E₁`    |
| `E₁ E₂ …`       | `Pr E₁ ; Pr E₂ ; …`                 |
| `E₁ │ E₂ │ …`   | `case sym of`<br> `first(E₁): Pr E₁ `<br> `first(E₂): Pr E₂`<br> `…`<br> `otherwise error` |

The procedure of the start symbol has to be called to recognize a sentence in the language.

For example, consider grammar `G₅`:

    A → a A c | b

Condition 1 requires `first(a b A c) ∩ first(b) = {}`, which holds. Condition 2 applies only for nullable nonterminals, but `A` is not nullable; both conditions are satisfied. Applying the above rules to `A` results in:

    procedure A
        case sym of
            'a': next ; A ; if sym = 'c' then next else error
            'b': next
        otherwise error

Above, we have already applied several simplifications. Generally useful transformations are:  

| Parser          | Simplified Parser                   |
|:----------------|:------------------------------------|
|<code>case sym of<br>    L: if sym ∈ L then S else error<br>    …</code>|<code>case sym of<br>    L: S<br>    …</code> |
|<code>while sym ∈ L do<br>    if sym ∈ L then S else error</code>|<code>while sym ∈ L do<br>    S</code>|
|<code>case sym of<br>    L₁: S₁<br>    L₂: S₂<br>    …<br> otherwise error</code>|<code>if sym ∈ L₁ then S₁<br> else if sym ∈ L₂ then S₂<br> …<br> else error<br> </code>

The implementation below of the parser for `G₅` consists of a single recursive parsing procedure `A`. The input is a sequence of characters stored in the global variable `src`. An index to the next symbol is maintained. An end-of-input symbol that does not occur otherwise is appended to the input. If that symbol is encountered prematurely, the parser exists from the recursion without consuming further input symbols. After returning from the recursion, there is a test that all input symbols have been processed:

In [ ]:
src: str; pos: int; sym: str

def nxt():
    global pos, sym
    if pos < len(src): sym, pos = src[pos], pos + 1
    else: sym = chr(0) # end of input symbol

def A(): # A → a A c | b
    if sym == 'a':
        nxt(); A();
        if sym == 'c': nxt()
        else: raise Exception("'c' expected at " + str(pos))
    elif sym == 'b': nxt()
    else: raise Exception("'a' or 'b' expected at " + str(pos))

def parse(s: str):
    global src, pos;
    src, pos = s, 0; nxt(); A()
    if sym != chr(0): raise Exception("unexpected characters at " + str(pos))

parse("aaabccc")

Now consider grammar `G₆` with:

    A → A a | b

Condition 1 for recursive descent parsing requires that `first(A a) ∩ first(b) = {}`:

    first(A a) = {b}
    first(b) = {b}

Hence, a recursive descent parser cannot be constructed. In general, a recursive descent parser cannot be constructed for any grammar with left-recursive productions. However, a grammar with left-recursive productions can be rewritten into an equivalent grammar with right-recursive productions. For `G₆:`

    A → A A'
    A' → a A' | ε

Consider grammar `G₃`:

    E → T | E + T
    T → F | T × F
    F → id | ( E )

An equivalent grammar that is not left-recursive is:

    E → T E'
    E' → + T E' | ε
    T → F T'
    T' → × F T' | ε
    F → id | ( E )

However, this grammar no longer reflects the structure of arithmetic expressions naturally. Using EBNF avoids that: the grammars are more natural and allow a simple construction of parsers.

For example, grammar `G₆` can be rewritten with EBNF without recursion at all:

    A → b {a}

For an EBNF grammar, the two conditions for recursive descent parsing are rephrased as follows:

| `E`           | `condition(E)`                               |
|:--------------|:---------------------------------------------|
| `[E₁]`        | `first(E₁) ∩ follow(E) = {}`                 |
| `{E₁}`        | `first(E₁) ∩ follow(E) = {}`                 |
| `E₁ E₂ …`     | `first(Eᵢ) ∩ first(Eᵢ₊₁ Eᵢ₊₂ …) = {}` for any nullable `Eᵢ`, provided `Eᵢ₊₁ Eᵢ₊₂ …` is not nullable |
| `E₁ │ E₂ │ …` | `first(Eᵢ) ∩ first(Eⱼ) = {}` for all `i ≠ j` |

For the production `A → b {a}`, the conditions are:

1. `first(b) ∩ first({a}) = {}` if `b` is nullable; as `b` is not nullable, the condition holds,
2. `first(a) ∩ follow({a}) = {}`, which holds as `first(a) = {a}` and `follow(A) = {}`.

As both conditions hold, a parser can be constructed:

    procedure A()
        if sym = 'b' then next else error
        while sym = 'a' do next

Here is an implementation in Python:

In [ ]:
src: str; pos: int; sym: str

def nxt():
    global pos, sym
    if pos < len(src): sym, pos = src[pos], pos+1
    else: sym = chr(0) # end-of-input symbol

def A(): # A → b {a}
    if sym == 'b': nxt()
    else: raise Exception("'b' expected at " + str(pos))
    while sym == 'a': nxt()

def parse(s: str):
    global src, pos;
    src, pos = s, 0; nxt(); A()
    if sym != chr(0): raise Exception("unexpected characters at " + str(pos))

parse("baa")

Let us construct a parser for regular expressions. The symbols are characters:

    expression  →  term { '|' term }
    term  →  factor { factor }
    factor  →  atom [ '*' | '+' | '?' ]
    atom  →  plainchar | escapedchar | '(' expression ')'
    plainchar  →  ' ' | '!' | '"' | '#' | '$' | '%' | '&' | '\'' | ',' | '-' | '.' | '/' |
         '0' | '1' | '2' | '3' | '4' | '5' | '6' | '7' | '8' | '9' | ':' | ';' | '<' | '=' | '>' | 
         '@' | 'A' | 'B' | 'C' | 'D' | 'E' | 'F' | 'G' | 'H' | 'I' | 'J' | 'K' | 'L' | 'M' | 'N' | 'O' |
         'P' | 'Q' | 'R' | 'S' | 'T' | 'U' | 'V' | 'W' | 'X' | 'Y' | 'Z' | '[' | ']' | '^' | '_' |
         '`' | 'a' | 'b' | 'c' | 'd' | 'e' | 'f' | 'g' | 'h' | 'i' | 'j' | 'k' | 'l' | 'm' | 'n' | 'o' |
         'p' | 'q' | 'r' | 's' | 't' | 'u' | 'v' | 'w' | 'x' | 'y' | 'z' | '{' | '}' | '~'
    escapedchar  → '\' ( '(' | ')' | '*' | '+' | '?' | '\' | '|' )


We need to check the conditions for recursive descend parsing:

- For `term { '|' term }`:   `term` is not nullable, condition holds.
- For `{ '|' term }`:  `first('|' term) ∩ follow({ '|' term }) = {}`, condition holds.
- For `'|' term`:  `'|'` is not nullable, condition holds.
- For `factor { factor }`:  `factor` is not nullable, condition holds.
- For `{ factor }`:  `first(factor) ∩ follow({ factor }) = {}`, condition holds.
- For `atom [ '*' | '+' | '?' ]`:  `atom` is not nullable, condition holds.
- For `[ '*' | '+' | '?' ]`:  `first('*' | '+' | '?') ∩ follow([ '*' | '+' | '?' ]) = {}`, condition holds.
- For `'*' | '+' | '?'`:  `first` of any two of `'*'`, `'+'`, `'?'` are disjoint, conditions hold.
- For `plainchar | escapedchar | '(' expression ')'`:  `first` of any two of `plainchar`, `escapedchar`, `'(' expression ')'` are disjoint, conditions hold.
- For `'(' expression ')'`: neither `(` nor `expression` is nullable, conditions hold.
- For `' ' | '!' | '"' | '#' | '$' | '%' | ...`:  `first` of any two are disjoint, conditions hold.
- For `'\' ( '(' | ')' | '*' | '+' | '?' | '\' | '|' )`:  `'\'` is not nullable, condition holds.
- For `'(' | ')' | '*' | '+' | '?' | '\' | '|'`:  `first` of any two of `'('`, `')'`, `'*'`, `'+'`, `'?'`, `'\\'`, `'|'` are disjoint, conditions hold.

As the symbols are characters, the implementation uses strings rather than sets of characters for the `first` sets.

In [ ]:
PlainChars = ' !"#$%&\',-./0123456789:;<=>@ABCDEFGHIJKLMNO' + \
                       'PQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{}~'
EscapedChars = '()*+?\\|'
FirstFactor = PlainChars + '\\('

src: str; pos: int; sym: str

def nxt():
    global pos, sym
    if pos < len(src): sym, pos = src[pos], pos+1
    else: sym = chr(0) # end-of-input symbol

def expression(): # expression → term { '|' term }
    term()
    while sym == '|': nxt(); term()

def term(): # term → factor { factor } 
    factor()
    while sym in FirstFactor: factor()

def factor(): # factor → atom [ '*' | '+' | '?' ]
    atom()
    if sym in '*+?': nxt()

def atom(): # atom → plainchar | escapedchar | '(' expression ')'
    if sym in PlainChars: nxt()
    elif sym == '\\':
        nxt()
        if sym in EscapedChars: nxt()
        else: raise Exception("invalid escaped character at " + str(pos))
    elif sym == '(':
        nxt(); expression()
        if sym == ')': nxt()
        else: raise Exception("')' expected at " + str(pos))
    else: raise Exception("invalid character at " + str(pos))

def parse(s: str):
    global src, pos;
    src, pos = s, 0; nxt(); expression()
    if sym != chr(0): raise Exception("unexpected character at " + str(pos))

#parse("a\$") # Exception: invalid escaped character at 3
#parse("a(b") # Exception: ')' expected at 3
#parse("a(" + chr(5) + ")") # invalid character at 3
#parse("a" + chr(5)) # unexpected character at 2
parse("(a*)*abcc")